# Limpieza y Preparación de Datos

## Objetivos de Aprendizaje
- Identificar y manejar datos faltantes
- Detectar y tratar valores atípicos (outliers)
- Eliminar duplicados y normalizar datos
- Validar la calidad de los datos
- Aplicar técnicas de feature engineering básico

## Requisitos
- Python 3.8+
- pandas
- numpy
- matplotlib
- seaborn

In [ ]:
# Instalación de dependencias
import sys
!{sys.executable} -m pip install pandas numpy matplotlib seaborn scipy -q

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configuración
pd.set_option('display.max_columns', None)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("Librerías cargadas correctamente")

## 1. Crear Dataset con Problemas de Calidad

Vamos a crear un dataset con varios problemas típicos de calidad.

In [ ]:
# Crear dataset con problemas
np.random.seed(42)

n_records = 1000

df_dirty = pd.DataFrame({
    'id': range(1, n_records + 1),
    'nombre': [
        np.random.choice(['  Juan  ', 'MARÍA', 'pedro', 'Ana López', None, 'Carlos', 'Laura '])
        for _ in range(n_records)
    ],
    'email': [
        np.random.choice([
            'juan@email.com', 'maria@test.com', 'invalido', None, 'pedro@example.com',
            'ANA@EMAIL.COM', 'carlos@test', '  laura@email.com  '
        ])
        for _ in range(n_records)
    ],
    'edad': np.random.choice([25, 30, None, -5, 35, 150, 28, 40], n_records),
    'salario': np.random.choice(
        [30000, 45000, None, -1000, 60000, 1000000, 55000, 70000],
        n_records
    ),
    'fecha_registro': pd.date_range('2020-01-01', periods=n_records, freq='D'),
    'ciudad': np.random.choice(['Madrid', 'madrid', 'BARCELONA', 'Barcelona', None], n_records),
    'pais': np.random.choice(['España', 'spain', 'ESPAÑA', None], n_records)
})

# Agregar algunos duplicados
df_dirty = pd.concat([df_dirty, df_dirty.iloc[:10]], ignore_index=True)

print(f"Dataset creado con {len(df_dirty)} registros")
print(f"\nPrimeras filas:")
df_dirty.head(10)

## 2. Análisis Exploratorio de Problemas de Calidad

In [ ]:
# Información general
print("=== INFORMACIÓN DEL DATASET ===")
print(f"\nShape: {df_dirty.shape}")
print(f"\nColumnas: {df_dirty.columns.tolist()}")
print(f"\nTipos de datos:")
print(df_dirty.dtypes)

print(f"\n=== ESTADÍSTICAS DESCRIPTIVAS ===")
df_dirty.describe()

In [ ]:
# Análisis de valores nulos
print("=== VALORES NULOS ===")
null_counts = df_dirty.isnull().sum()
null_percentages = (null_counts / len(df_dirty) * 100).round(2)

null_summary = pd.DataFrame({
    'Nulos': null_counts,
    'Porcentaje': null_percentages
})

print(null_summary[null_summary['Nulos'] > 0])

# Visualización
plt.figure(figsize=(10, 6))
null_summary['Porcentaje'].plot(kind='bar', color='coral')
plt.title('Porcentaje de Valores Nulos por Columna')
plt.ylabel('Porcentaje (%)')
plt.xlabel('Columna')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Análisis de duplicados
print("=== ANÁLISIS DE DUPLICADOS ===")
duplicates = df_dirty.duplicated()
print(f"\nTotal de filas duplicadas: {duplicates.sum()}")
print(f"Porcentaje: {(duplicates.sum() / len(df_dirty) * 100):.2f}%")

if duplicates.sum() > 0:
    print(f"\nEjemplos de duplicados:")
    print(df_dirty[duplicates].head())

## 3. Limpieza de Datos Paso a Paso

In [ ]:
# Crear copia para limpiar
df_clean = df_dirty.copy()

print(f"Dataset original: {len(df_dirty)} filas")
print(f"Iniciando limpieza...")

### 3.1. Eliminar Duplicados

In [ ]:
# Eliminar duplicados completos
before = len(df_clean)
df_clean = df_clean.drop_duplicates()
after = len(df_clean)

print(f"Duplicados eliminados: {before - after}")
print(f"Registros restantes: {after}")

### 3.2. Limpiar Columnas de Texto

In [ ]:
# Limpiar nombres
print("=== LIMPIEZA DE NOMBRES ===")
print("\nAntes:")
print(df_clean['nombre'].value_counts())

# Aplicar limpieza
df_clean['nombre'] = df_clean['nombre'].str.strip().str.title()

print("\nDespués:")
print(df_clean['nombre'].value_counts())

In [ ]:
# Limpiar emails
print("=== LIMPIEZA DE EMAILS ===")

# Normalizar: quitar espacios y convertir a minúsculas
df_clean['email'] = df_clean['email'].str.strip().str.lower()

# Validar formato de email
import re
email_pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'

def is_valid_email(email):
    if pd.isna(email):
        return False
    return bool(re.match(email_pattern, email))

df_clean['email_valido'] = df_clean['email'].apply(is_valid_email)

print(f"\nEmails válidos: {df_clean['email_valido'].sum()}")
print(f"Emails inválidos: {(~df_clean['email_valido']).sum()}")

print("\nEjemplos de emails inválidos:")
print(df_clean[~df_clean['email_valido']]['email'].value_counts())

In [ ]:
# Normalizar ciudades y países
print("=== NORMALIZACIÓN DE CIUDADES Y PAÍSES ===")

# Ciudades
df_clean['ciudad'] = df_clean['ciudad'].str.strip().str.title()
print("\nCiudades únicas:")
print(df_clean['ciudad'].value_counts())

# Países
df_clean['pais'] = df_clean['pais'].str.strip().str.title()
print("\nPaíses únicos:")
print(df_clean['pais'].value_counts())

### 3.3. Limpiar Datos Numéricos

In [ ]:
# Analizar edad
print("=== ANÁLISIS DE EDAD ===")
print(f"\nEstadísticas:")
print(df_clean['edad'].describe())

# Visualizar distribución
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histograma
df_clean['edad'].hist(bins=30, ax=axes[0], color='skyblue')
axes[0].set_title('Distribución de Edad')
axes[0].set_xlabel('Edad')
axes[0].set_ylabel('Frecuencia')

# Boxplot
df_clean.boxplot(column='edad', ax=axes[1])
axes[1].set_title('Boxplot de Edad')
axes[1].set_ylabel('Edad')

plt.tight_layout()
plt.show()

# Identificar valores problemáticos
print(f"\nValores negativos: {(df_clean['edad'] < 0).sum()}")
print(f"Valores > 100: {(df_clean['edad'] > 100).sum()}")
print(f"Valores nulos: {df_clean['edad'].isnull().sum()}")

In [ ]:
# Limpiar edad: establecer límites razonables
print("=== LIMPIEZA DE EDAD ===")

# Convertir valores fuera de rango a NaN
df_clean.loc[(df_clean['edad'] < 18) | (df_clean['edad'] > 100), 'edad'] = np.nan

# Imputar valores faltantes con la mediana
mediana_edad = df_clean['edad'].median()
df_clean['edad'].fillna(mediana_edad, inplace=True)

print(f"\nValores imputados con mediana: {mediana_edad}")
print(f"\nNueva distribución:")
print(df_clean['edad'].describe())

In [ ]:
# Limpiar salario
print("=== ANÁLISIS DE SALARIO ===")
print(f"\nEstadísticas:")
print(df_clean['salario'].describe())

# Identificar outliers con IQR
Q1 = df_clean['salario'].quantile(0.25)
Q3 = df_clean['salario'].quantile(0.75)
IQR = Q3 - Q1

limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

print(f"\nRango IQR:")
print(f"  Límite inferior: {limite_inferior:,.2f}")
print(f"  Límite superior: {limite_superior:,.2f}")

outliers = df_clean[(df_clean['salario'] < limite_inferior) | (df_clean['salario'] > limite_superior)]
print(f"\nOutliers detectados: {len(outliers)}")

# Visualizar
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
df_clean['salario'].hist(bins=30, color='lightgreen')
plt.title('Distribución de Salario')
plt.xlabel('Salario')
plt.ylabel('Frecuencia')

plt.subplot(1, 2, 2)
df_clean.boxplot(column='salario')
plt.title('Boxplot de Salario')

plt.tight_layout()
plt.show()

In [ ]:
# Limpiar salario
print("=== LIMPIEZA DE SALARIO ===")

# Convertir valores negativos y outliers extremos a NaN
df_clean.loc[(df_clean['salario'] < 0) | (df_clean['salario'] > 200000), 'salario'] = np.nan

# Imputar con la mediana
mediana_salario = df_clean['salario'].median()
df_clean['salario'].fillna(mediana_salario, inplace=True)

print(f"\nValores imputados con mediana: ${mediana_salario:,.2f}")
print(f"\nNueva distribución:")
print(df_clean['salario'].describe())

### 3.4. Validación de Datos Limpios

In [ ]:
# Crear reporte de calidad
print("=== REPORTE DE CALIDAD DE DATOS ===")

quality_report = pd.DataFrame({
    'Columna': df_clean.columns,
    'Tipo': df_clean.dtypes.values,
    'Nulos': df_clean.isnull().sum().values,
    '% Nulos': (df_clean.isnull().sum() / len(df_clean) * 100).values.round(2),
    'Únicos': [df_clean[col].nunique() for col in df_clean.columns]
})

print(quality_report)

In [ ]:
# Comparación antes y después
print("=== COMPARACIÓN ANTES Y DESPUÉS ===")
print(f"\nRegistros originales: {len(df_dirty)}")
print(f"Registros limpios: {len(df_clean)}")
print(f"Registros eliminados: {len(df_dirty) - len(df_clean)}")

print(f"\nValores nulos originales: {df_dirty.isnull().sum().sum()}")
print(f"Valores nulos después: {df_clean.isnull().sum().sum()}")

## 4. Feature Engineering Básico

In [ ]:
# Crear nuevas características
print("=== FEATURE ENGINEERING ===")

# Categoría de edad
df_clean['categoria_edad'] = pd.cut(
    df_clean['edad'],
    bins=[0, 25, 35, 50, 100],
    labels=['18-25', '26-35', '36-50', '51+']
)

# Categoría de salario
df_clean['categoria_salario'] = pd.cut(
    df_clean['salario'],
    bins=[0, 40000, 60000, 100000],
    labels=['Bajo', 'Medio', 'Alto']
)

# Extraer componentes de fecha
df_clean['año_registro'] = df_clean['fecha_registro'].dt.year
df_clean['mes_registro'] = df_clean['fecha_registro'].dt.month
df_clean['dias_desde_registro'] = (datetime.now() - df_clean['fecha_registro']).dt.days

print("\nNuevas características creadas:")
print(df_clean[[
    'edad', 'categoria_edad',
    'salario', 'categoria_salario',
    'fecha_registro', 'dias_desde_registro'
]].head())

## 5. Exportar Datos Limpios

In [ ]:
# Guardar dataset limpio
output_path = '../../datasets/processed/datos_limpios.csv'
df_clean.to_csv(output_path, index=False)

print(f"Dataset limpio guardado en: {output_path}")
print(f"\nShape final: {df_clean.shape}")
print(f"\nColumnas finales:")
print(df_clean.columns.tolist())

## Resumen y Mejores Prácticas

### Proceso de Limpieza:
1. **Análisis exploratorio**: Identificar problemas de calidad
2. **Eliminar duplicados**: Asegurar unicidad de registros
3. **Normalizar texto**: Estandarizar formato de strings
4. **Validar datos**: Verificar rangos y formatos
5. **Manejar outliers**: Detectar y tratar valores atípicos
6. **Imputar valores faltantes**: Estrategia según el contexto
7. **Feature engineering**: Crear características derivadas

### Mejores Prácticas:
- Siempre trabajar en una copia del dataset original
- Documentar cada paso de limpieza
- Validar resultados después de cada transformación
- Considerar el contexto del negocio para decisiones de limpieza
- Automatizar el proceso con funciones reutilizables
- Mantener registro de cambios (auditoría)

### Técnicas de Imputación:
- **Media/Mediana**: Para datos numéricos
- **Moda**: Para datos categóricos
- **Forward/Backward fill**: Para series temporales
- **Interpolación**: Para datos secuenciales
- **Valor constante**: Cuando tiene sentido de negocio

### Recursos Adicionales:
- [Pandas Data Cleaning](https://pandas.pydata.org/docs/user_guide/missing_data.html)
- [Data Quality Best Practices](https://www.dataversity.net/data-quality-best-practices/)
- [Feature Engineering Guide](https://www.kaggle.com/learn/feature-engineering)